In [70]:
import hashlib
import base64
import numpy as np
from PIL import Image
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.backends import default_backend
from cryptography.exceptions import InvalidSignature

class CryptoSteganography(object):
   

    def __init__(self, key):
        # Create a sha256 hash from the informed string key
        self.key = hashlib.sha256(key.encode()).digest()

    def generate_ecc_key_pair(self):
        private_key = ec.generate_private_key(ec.SECP256R1(), default_backend())

        public_key = private_key.public_key()
        return private_key, public_key

    def encode_message(self, message):

        private_key, public_key = self.generate_ecc_key_pair()

        signature = private_key.sign(message.encode(), ec.ECDSA(hashes.SHA256()))
        return public_key, signature

    def decode_message(self, public_key, signature, message):

        try:
            public_key.verify(signature, message.encode(), ec.ECDSA(hashes.SHA256()))
            return True
        except InvalidSignature:
            return False

    def hide(self, input_filename, output_filename, data):
        
        # ECC Encoding
        public_key, signature = self.encode_message(data)
        public_key_bytes = public_key.public_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo
        )
        signature_bytes = signature
    
        # Serialize secret data
        data_bytes = data.encode()
        

        concatenated_data = public_key_bytes + signature_bytes + data_bytes

        # Convert concatenated data to base64 string
        concatenated_data_base64 = base64.b64encode(concatenated_data).decode()

        input_image = Image.open(input_filename)
        width, height = input_image.size

        # Convert the data to binary
        binary_data = ''.join(format(ord(char), '08b') for char in concatenated_data_base64)

        # Embed data into image using PVD algorithm
        img_data = np.array(input_image)
        data_index = 0
        if len(img_data.shape) == 3:  # RGB image
            for y in range(height):
                for x in range(width):
                    if data_index < len(binary_data):
                        r, g, b = img_data[y, x]
                        img_data[y, x] = (r, g, (b & 0xFE) | int(binary_data[data_index]))
                        data_index += 1
                    else:
                        break
        else:  # B/W image
            for y in range(height):
                for x in range(width):
                    if data_index < len(binary_data):
                        pixel_value = img_data[y, x]
                        img_data[y, x] = (pixel_value & 0xFE) | int(binary_data[data_index])
                        data_index += 1
                    else:
                        break

        output_image = Image.fromarray(img_data)
        output_image.save(output_filename)

    def retrieve(self, input_image_file):
        
        input_image = Image.open(input_image_file)
        width, height = input_image.size


        binary_data = ''
        for y in range(height):
            for x in range(width):
                pixel_value = input_image.getpixel((x, y))
                if isinstance(pixel_value, tuple):  # RGB image
                    r, g, b = pixel_value
                    binary_data += str(b & 1)
                else:  # B/W image
                    binary_data += str(pixel_value & 1)

        # Convert binary data to base64 string
        concatenated_data_base64 = ''.join(chr(int(binary_data[i:i+8], 2)) for i in range(0, len(binary_data), 8))

        # Decode base64 string
        concatenated_data = base64.b64decode(concatenated_data_base64.encode())

        # Retrieve the public key, signature, and secret data
        public_key_bytes = concatenated_data[:173]  # Assuming length of public key is 173 bytes
        signature_bytes = concatenated_data[173:237]  # Assuming signature length is 64 bytes

        # Extract secret data
        secret_data = concatenated_data[237:]

        return public_key_bytes, signature_bytes, secret_data

# Example usage:
if __name__ == "__main__":
    key = "YourSecretKey"
    input_image = "textureB.png"
    output_image = "outt.png"
    data_to_hide = input("Enter the secret data: ")
    crypto_steganography = CryptoSteganography(key)
    crypto_steganography.hide(input_image, output_image, data_to_hide)
    
    retrieved_public_key, retrieved_signature, retrieved_secret_data = crypto_steganography.retrieve(output_image)
    print("Retrieved public key:", retrieved_public_key)
    print("Retrieved signature:", retrieved_signature)
    print("Retrieved secret data:", retrieved_secret_data)


Enter the secret data: tatdce_trdae_cadeatattarscs_tttddteettetaddctcseettaetcttr_artarscaedeeaaetsresdaactaetaeartteattacderae_tsctaeteea_t_ttea_tcttrctersrttas_cdtettcadcreetd_desasasaddecrea_ettssacsereear_aatttcesde_eteddtedasetsctedesaacasttrtrcsatatatdttct__tteeatacttcaedcasecs_tc_areeeataaersca_eedrertesat_reedtdcdttcttteeeedsed_taeetsserctetcce_etaaa_addseceatdtedasatcatteesdtrr_ae_atac_aaa_rte_desecttcad_rcetrat__aade_drtdrdseesttrsaeae__raeaa_eetdcccaaeerts__ar_tcaessedst_t_cc_s__atrttaatartt_asraaerressrtartaasstescdd_ccsstassdrd_tettaedeeaaeraceetedared_aedrcaastsaatasttedraeee_taarca_ctdaatact__ateartde_ae_as_atcs_r_dtater_atrtec_cretatracrr_atecstess_aaeaateardcasaed__s_cs_aea_dtestasdt_tettesreetteetetsrcsc_erarcre_tat_caeaaaarttsaae_ceetese_eearaarettctaecsretts_tetre_s_dadte_ca_sas_aaacear_cdassassred_rra_tdaetcetedddcseacetrees_aetaeaa_serassterdsttertactcetedrraeeeetrscaarddt_eteasctddtatesatt_aesdtaedatseeat_sedeccaeac_te_casactrae_treretartt_steccers_sdecteet_adetd

Retrieved public key: b'-----BEGIN PUBLIC KEY-----\nMFkwEwYHKoZIzj0CAQYIKoZIzj0DAQcDQgAEEGANAC8+4fZ9sYFmE1ufe008h6FZ\nWFBtEZsrxWdnkVtB9bUWcqXfIPwwWNur2mgVSJCQYmhxR4osqHnF3KmZuQ==\n-----END PUBLIC KEY-'
Retrieved signature: b'----\n0E\x02 M\xed\x0f\xa5|\xac\xc4SdT\xa4\t\x18\x0b$\xdb\xa7\xffS\xd4=\xd7m\xed\xfa\xdb#\xe3ty[-\x02!\x00\x95@6=\xa73\xb8MwVR\xf1|\xce\xb6\x88L\x0fW\x8e'
Retrieved secret data: b'4\xc3\xd1\x82A\xc7+\xbf\x10C\xac\xf3tatdce_trdae_cadeatattarscs_tttddteettetaddctcseettaetcttr_artarscaedeeaaetsresdaactaetaeartteattacderae_tsctaeteea_t_ttea_tcttrctersrttas_cdtettcadcreetd_desasasaddecrea_ettssacsereear_aatttcesde_eteddtedasetsctedesaacasttrtrcsatatatdttct__tteeatacttcaedcasecs_tc_areeeataaersca_eedrertesat_reedtdcdttcttteeeedsed_taeetsserctetcce_etaaa_addseceatdtedasatcatteesdtrr_ae_atac_aaa_rte_desecttcad_rcetrat__aade_drtdrdseesttrsaeae__raeaa_eetdcccaaeerts__ar_tcaessedst_t_cc_s__atrttaatartt_asraaerressrtartaasstescdd_ccsstassdrd_tettaedeeaaeraceetedared_aedrcaasts

In [66]:
from PIL import Image
import numpy as np

def psnr(original_image, modified_image):
    # Convert images to grayscale
    original_image = original_image.convert('L')
    modified_image = modified_image.convert('L')

    # Convert images to numpy arrays
    original_array = np.array(original_image)
    modified_array = np.array(modified_image)

    # Calculate MSE (Mean Squared Error)
    mse = np.mean((original_array - modified_array) ** 2)

    # Maximum possible pixel value
    max_pixel = 255.0

    # Calculate PSNR (Peak Signal-to-Noise Ratio)
    psnr = 20 * np.log10(max_pixel / np.sqrt(mse))
    return psnr

# Load the original and modified images
original_image = Image.open('cellcolony.png')
modified_image = Image.open('outt.png')

# Resize the modified image to match the dimensions of the original image
modified_image = modified_image.resize(original_image.size)

# Calculate PSNR
psnr_value = psnr(original_image, modified_image)
print("PSNR:", psnr_value)


PSNR: inf


C:\Users\RITUJA\AppData\Local\Temp\ipykernel_2676\1186901667.py:20: RuntimeWarning: divide by zero encountered in double_scalars
  psnr = 20 * np.log10(max_pixel / np.sqrt(mse))


In [71]:
from PIL import Image
import numpy as np

def psnr(original_image, modified_image):
    # Convert images to grayscale
    original_image = original_image.convert('L')
    modified_image = modified_image.convert('L')

    # Convert images to numpy arrays
    original_array = np.array(original_image)
    modified_array = np.array(modified_image)

    # Calculate MSE (Mean Squared Error)
    mse = np.mean((original_array - modified_array) ** 2)

    # Maximum possible pixel value
    max_pixel = 255.0

    # Calculate PSNR (Peak Signal-to-Noise Ratio)
    psnr = 20 * np.log10(max_pixel / np.sqrt(mse))
    return mse, psnr

# Load the original and modified images
original_image = Image.open('textureB.png')
modified_image = Image.open('outt.png')

# Resize the modified image to match the dimensions of the original image
modified_image = modified_image.resize(original_image.size)

# Calculate MSE and PSNR
mse_value, psnr_value = psnr(original_image, modified_image)
print("MSE:", mse_value)
print("PSNR:", psnr_value)


MSE: 0.050141334533691406
PSNR: 61.1288447215307


In [72]:
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage import img_as_float

def calculate_ssim(original_image, modified_image):
    # Convert images to floating point format
    original_image = img_as_float(original_image)
    modified_image = img_as_float(modified_image)
    
    # Calculate SSIM for each channel and take the average
    ssim_values = []
    for channel in range(original_image.shape[-1]):
        ssim_channel = ssim(original_image[:, :, channel], modified_image[:, :, channel], data_range=modified_image.max() - modified_image.min())
        ssim_values.append(ssim_channel)
    
    # Average SSIM values across channels
    avg_ssim = np.mean(ssim_values)
    
    return avg_ssim

# Example usage:
if __name__ == "__main__":
    # Load the original and modified images
    original_image = Image.open('textureB.png').convert('RGB')
    modified_image = Image.open('outt.png').convert('RGB')

    # Resize the modified image to match the dimensions of the original image
    modified_image = modified_image.resize(original_image.size)

    # Calculate SSIM
    ssim_value = calculate_ssim(original_image, modified_image)
    print("SSIM:", ssim_value)


SSIM: 0.9999381812855009
